#Objective is to predict fraud 


In [1]:
#load libraries
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc, recall_score, precision_score, f1_score, confusion_matrix, recall_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [3]:
#load data
df = pd.read_csv('transactions.csv', sep=",")

In [5]:
df.head()

,Unnamed: 0,CURRENCY,AMOUNT,TRAN_STATE,TRAN_CREATED_DATE,MERCHANT_CATEGORY,MERCHANT_COUNTRY,ENTRY_METHOD,USER_ID,TYPE,...,COUNTRY,BIRTH_YEAR,KYC,FAILED_SIGN_IN_ATTEMPTS,CODE,NAME,CODE3,NUMCODE,PHONECODE,IS_FRAUDSTER_N
0,0,GBP,175,COMPLETED,46:20.3,cafe,GBR,cont,8f99c254-7cf2-4e35-b7e4-53804d42445d,CARD_PAYMENT,...,GB,1962.0,PASSED,0.0,GB,United Kingdom,gBR,826.0,44.0,0
1,1,EUR,2593,COMPLETED,38:47.2,bar,AUS,cont,ed773c34-2b83-4f70-a691-6a7aa1cb9f11,CARD_PAYMENT,...,IE,1995.0,PASSED,0.0,IE,Ireland,iRL,372.0,353.0,0
2,2,EUR,1077,COMPLETED,34:39.7,NaN,CZE,cont,eb349cc1-e986-4bf4-bb75-72280a7b8680,CARD_PAYMENT,...,CZ,1985.0,PASSED,0.0,CZ,Czech Republic,CZE,203.0,420.0,0
3,3,GBP,198,COMPLETED,45:50.6,supermarket,GBR,cont,dc78fbc4-c936-45d3-a813-e2477ac6d74b,CARD_PAYMENT,...,GB,1974.0,PASSED,0.0,GB,United Kingdom,gBR,826.0,44.0,0
4,4,EUR,990,COMPLETED,45:32.7,NaN,FRA,cont,32958a5c-2532-42f7-94f9-127f2a812a55,CARD_PAYMENT,...,FR,1991.0,PASSED,0.0,FR,France,FRA,250.0,33.0,0


In [4]:
#data summary
df.describe()

,Unnamed: 0,AMOUNT,AMOUNT_USD,EXPONENT,HAS_EMAIL,BIRTH_YEAR,FAILED_SIGN_IN_ATTEMPTS,NUMCODE,PHONECODE,IS_FRAUDSTER_N
count,57748.00000,5.774800e+04,5.774800e+04,57748.000000,55262.000000,55262.00000,55262.000000,55243.000000,55243.000000,57748.0
mean,28873.50000,3.955198e+04,5.993035e+06,2.009005,0.995729,1984.93795,0.008342,651.194341,105.287150,0.0
std,16670.55601,1.433153e+06,2.695487e+08,0.241621,0.065210,9.73848,0.127983,229.477925,128.115641,0.0
min,0.00000,0.000000e+00,0.000000e+00,0.000000,0.000000,1930.00000,0.000000,40.000000,30.000000,0.0
25%,14436.75000,5.290000e+02,6.010000e+02,2.000000,1.000000,1981.00000,0.000000,440.000000,44.000000,0.0
50%,28873.50000,1.545000e+03,1.638000e+03,2.000000,1.000000,1987.00000,0.000000,826.000000,44.000000,0.0
75%,43310.25000,5.100000e+03,5.613250e+03,2.000000,1.000000,1992.00000,0.000000,826.000000,47.000000,0.0
max,57747.00000,1.775452e+08,2.373411e+10,8.000000,1.000000,2000.00000,6.000000,826.000000,590.000000,0.0


In [ ]:
#check for missing data


In [ ]:
#fix data imbalance

In [ ]:
#split categorical data

In [ ]:
#split cts data

In [ ]:
#combine data

In [ ]:
#perform modeling

In [ ]:
#build AUC